# First steps with quantum circuits
This contains first steps with quantum circuits taken out of the coursera course "Practical Quantum computing with IBM Quiskit for Beginners" offered by Packt. The course is based on some older version of Qiskit and I decide for myself to update the content to a recent Qiskit version. I use this as a formulary to look up the properties of simple gates. 

## Installation

In [ ]:
from IPython.display import display, Markdown
import ipywidgets as widgets
import qiskit as q
import qiskit_aer as aer
import math
print(f"Quiskit Version: {q.__version__}")
print(f"Quiskit Aer Version: {aer.__version__}")

Let's create a helper function which displays all the data for the course.

In [ ]:
def ShowSimpleGate(qcircuit, title = '### Quantum Gate', showanimation=False):
    '''
    Calculates  a combination of simple quantum gates and returns an object to display
    Inputs:
        qcircuit:  a qiskit quantum circuit
        showanimation: Boolean. Set to true if you want to see an animated arrow in the bloch sphere (only works for 1qbit systems
    Outputs:
        out: An ipywidget gridspeclayout object you can display
    '''
    ###############################################################
    # Prepare Widget with the grid
    ###############################################################
    # Make a 2x2 output grid and initialize each element with widgets.Output()
    rows=2
    cols=2
    out = widgets.GridspecLayout(rows, cols, layout={
        'width': '60%',
        'align_items': 'flex-start',
        'justify_items': 'flex-start'
        })
    layout = {'width': '100%', 'align_items': 'flex-start', 'justify_items': 'flex-start'}
    [[out.__setitem__((i,j), widgets.Output(layout = layout)) for j in range(cols)] for i in range(rows)]

    ###############################################################
    # Get the text out of the circuit for the description
    ###############################################################
      
    annotation_parts = [title + '\n']
    for gate in qc.data:
        annotation_parts.append('- ' + str(gate.operation.name) + '(')
        for i, bit in enumerate(gate.qubits):
            annotation_parts.append(str(qc.find_bit(bit).index))
            if i==len(gate.qubits)-1:
                annotation_parts.append(')\n')
            else:
                annotation_parts.append(', ')
    annotation = "".join(annotation_parts)

    # top-left box with circuit description and scheme
    out[0,0].append_display_data(Markdown(annotation))
    out[0,0].append_display_data(qc.draw('mpl'))


    ##################################################################
    # Simulation
    #################################################################
    
    # Creating the simulator for the statevector (this is old style  to use the method shown in the course
    # Statevector and unitary matrix actually do not need a simulation run - see example below
    backend = aer.AerSimulator(method='automatic')
    #transpiling is especially important for newer versions of aer (rearranging the circuit to the given platform, here it's only simulation)
    qc_transpiled = q.transpile(qc, backend)
    # add a measurement because we need the state vector later on
    qc_transpiled.save_statevector()

    #Run the circuit
    job = backend.run(qc_transpiled)
    result = job.result()
    statevector = result.get_statevector()

    # put bloch sphere in top-right box
    
    # Animation is time-consuming (thus set to False by default) 
    # there is a deprecation warning, but it still seems to work fine for circuits with 1 qubit
    if showanimation:
        out[0,1].append_display_data(q.visualization.visualize_transition(qc))
    else: 
        # Draw bloch sphere
        out[0,1].append_display_data(q.visualization.plot_bloch_multivector(statevector))
    
    # Histogram in bottom-right box
    counts = result.get_counts()
    out[1,1].append_display_data(q.visualization.plot_histogram(counts))
    
    # put transition matrix and state in bottom-left box
    
    # To get the unitary matrix, you do not need a simlation. quantum_info provides all information
    out[1,0].append_display_data(q.visualization.array_to_latex(q.quantum_info.Operator(qc), prefix='\\text{Unitary matrix} = '))
    out[1,0].append_display_data(q.visualization.array_to_latex(q.quantum_info.Statevector(qc), prefix='\\text{State vector} = '))

    return out
                                 
                     
    

qc = q.QuantumCircuit(1)
qc.x(0)
out = ShowSimpleGate(qc, title = '### A simple circuit - Pauli x-gate')
display(out)

## Other simple gates shown in the lecture


In [ ]:
qc = q.QuantumCircuit(1)
qc.h(0)
# uncomment the next line to see the applying the gate twice returns to the origin
# qc.h(0)
out = ShowSimpleGate(qc, title = '### Hadamard gate \n The hadamard gate is used to create a superposition.\
    It performs a rotation of 180° around the XZ-axis(the [1,0,1] axis) in the Bloch Sphere.')
display(out)

qc = q.QuantumCircuit(1)
qc.y(0)
out = ShowSimpleGate(qc, title = '### Y-gate \n The Y-gate is a bit and phase flip gate. It rotates by 180° around the y-axis.')
display(out)

qc = q.QuantumCircuit(1)
qc.z(0)
out = ShowSimpleGate(qc, title = '### Z-gate \n The z-gate is the phase-flip gate and rotates by 180° around the z-axis.')
display(out)

qc = q.QuantumCircuit(1)
qc.s(0)
out= ShowSimpleGate(qc, title = '### S gate \n This creates a phase shift on spin down')
display(out)

qc = q.QuantumCircuit(1)
qc.t(0)
out = ShowSimpleGate(qc, title = '### T gate \n  Gate performs a rotation by 45° around the z-axis in the Bloch Sphere in the counter-clockwise direction.')
display(out)





## One quibit gates with parameters

### RX gate (in the lecture called R$\phi$)
This gate performs an arbitrary rotation around the x-axis. The matrix is

$$ 
RX = \begin{pmatrix}
\cos{\theta/2} & -i \sin{\theta/2} \\ -i \sin{\theta/2} & \cos{\theta/2} 
\end{pmatrix}
$$

### U3 (or U) gate
This gate performs rotations according to the given parameters $\theta$, $\phi$ and $\lambda$:
   - $\theta$ is the angle of rotation around the X-axis.
   - $\phi$ is the angle of rotation around the Z-axis. 
   - $\lambda$ is the angle of rotation around the Y-axis. 
$$
U  = \begin{pmatrix}
\cos{\theta/2} &   -\exp{i\lambda}\sin{\theta/2} \\
\exp{i\phi}\sin{\theta/2} & \exp{i(\phi+\lambda)} \cos(\theta/2) 
\end{pmatrix}
$$
Note that we are following the qiskit definition of this gate. Special cases are:
 -  $U(\theta,−\pi/2,\pi/2) = RX(\theta)$
 -  $U(\theta,0,0) = RY(\theta)$


In [ ]:
# Rphi gate
theta = math.pi/4
qc = q.QuantumCircuit(1)
qc.rx(theta, 0)
out = ShowSimpleGate(qc, title = '### RX gate \n This example shows $\\phi = \\pi/4$')
display(out)
# U gate
theta = math.pi/4
phi = math.pi/4
lam = math.pi/4

qc = q.QuantumCircuit(1)
qc.u(theta, phi, lam, 0)
out = ShowSimpleGate(qc, title = '### U gate \n This example shows all angles equal to $\\pi/4$')
display(out)


## Multi-Qbit-Gates

### Tensor product introduction

Having two qubits $|a> = \left(\begin{array}{ll} a_0 \\ a_1\end{array}\right)$ and $|b> = \left(\begin{array}{ll} b_0 \\ b_1\end{array}\right)$, the tensor product is defined as
$$ 
|ba> = |b> \otimes |a> = \left[ \begin{array}{ll} b_0 \left(\begin{array}{ll} a_0 \\ a_1\end{array}\right) \\ b_1 \left(\begin{array}{ll} a_0 \\ a_1\end{array}\right) \end{array}\right] = \left[ \begin{array}{ll}b_0 a_0 \\ b_0 a_1 \\ b_1 a_0 \\ b_1 a_1 \end{array}\right]
$$

If you apply a gate U and then a gate V to a 2-qubit system, then the transformation matrix is
$$
T = V\otimes U = \left[ \begin{array}{ll} u_{00}\left[ \begin{array}{ll} v_{00} && v_{01} \\ v_{10} && v_{11}  \end{array} \right] 
&& u_{01} \left[ \begin{array}{ll} v_{00} && v_{01} \\ v_{10} && v_{11}  \end{array} \right]
\\ u_{10} \left[ \begin{array}{ll} v_{00} && v_{01} \\ v_{10} && v_{11}  \end{array} \right] 
&& u_{11}  \left[ \begin{array}{ll} v_{00} && v_{01} \\ v_{10} && v_{11}  \end{array} \right]
\end{array} \right] 
$$

Beware that Qiskit uses litle endian notation, whereas some textbooks and some Bots provide big endian. 

In [ ]:
qc = q.QuantumCircuit(2)
qc.cx(0,1)
out = ShowSimpleGate(qc, title = '### CNOT gate \n Controlled NOT gate')
display(out)

qc = q.QuantumCircuit(2)
qc.cz(0,1)
out = ShowSimpleGate(qc, title = '### CZ gate \n Controlled Z gate')
display(out)

qc = q.QuantumCircuit(2)
qc.cy(0,1)
out = ShowSimpleGate(qc, title = '### CY gate \n Controlled Y gate')
display(out)

qc = q.QuantumCircuit(2)
qc.cx(0,1)
qc.cx(1,0)
qc.cx(0,1)
# (or just call swap())
out = ShowSimpleGate(qc, title = '### Swap gate \n Swaps states')
display(out)
